# **Importing Important Libraries**

In [ ]:
# import tensorflow as tf
# import numpy as np


class ZerosFirstEpochOnesAfter(tf.keras.utils.Sequence):
    def __init__(self):
        self.is_epoch_0 = True

    def __len__(self):
        return 2

    def on_epoch_end(self):
      print('\non_epoch_end')
      self.is_epoch_0 = False

    def __getitem__(self, item):
        if self.is_epoch_0:
            print("\nFirst epoch")
            return np.zeros((160, 1)), np.zeros((160,))
        else:
            return np.ones((160, 1)), np.full((160,), 99)


class OnEpochEnd(tf.keras.callbacks.Callback):
  def __init__(self, callbacks):
    self.callbacks = callbacks

  def on_epoch_end(self, epoch, logs=None):
    for callback in self.callbacks:
      callback()

if __name__ == '__main__':
    model = tf.keras.Sequential()
    model.add(tf.keras.layers.Dense(1, input_dim=1, activation="softmax"))

    model.compile(
        optimizer='Adam',
        loss='binary_crossentropy',
        metrics=['accuracy']
    )
    seq = ZerosFirstEpochOnesAfter()
    
    model.fit(seq, epochs=3,callbacks=[OnEpochEnd([seq.on_epoch_end])]) #

In [3]:
import tensorflow as tf

In [4]:
tf.config.experimental.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [ ]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
if gpus:
    for gpu in gpus:
       tf.config.experimental.set_memory_growth(gpu,True)

In [5]:
gpus = tf.config.list_physical_devices('GPU')
print(gpus)
if gpus:
    for gpu in gpus:
        tf.config.experimental.set_virtual_device_configuration(gpu,[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=12288)])

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


In [6]:
import matplotlib.pyplot as plt
import numpy as np
import PIL

from tensorflow import keras 
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import pathlib 
import os 
import PIL
from PIL import Image

# **Loading and Preprocessing Data**

In [7]:
data_dir = pathlib.Path("dataset")
# print(len(list(data_dir.glob("*/*"))))


In [ ]:
trash = list(data_dir.glob('Can/1/*'))
PIL.Image.open(str(trash[0]))

In [8]:
batch_size = 16
image_height = 224
image_width = 224

In [ ]:
class_names=['Can','Paper','Plastic']

In [42]:
# with tf.device("CPU"):
train_ds = keras.utils.image_dataset_from_directory(
  data_dir,
  # validation_split=0.1,
  # subset="training",
  seed=123,
  image_size=(image_height, image_width),
  batch_size=batch_size,
  # class_names=class_names
  )


Found 62651 files belonging to 3 classes.


In [65]:
test_dir = pathlib.Path("dcrpi-167")

# with tf.device("CPU"):
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  seed=123,
  image_size=(image_height, image_width),
  # class_names = class_names,
  batch_size=batch_size)



Found 167 files belonging to 3 classes.


In [67]:
# Create a function to convert labels to one-hot encoding
num_classes = 3
def to_one_hot(x, y):
    num_classes = 3
    y_one_hot = tf.one_hot(y, num_classes)
    return x, y_one_hot

In [12]:
train_ds = train_ds.map(to_one_hot)

In [ ]:
# val_ds = val_ds.map(to_one_hot)

In [68]:
test_ds = test_ds.map(to_one_hot)

# **Configure Dataset Performance**

In [ ]:
AUTOTUNE = tf.data.AUTOTUNE

train_ds = train_ds.prefetch(buffer_size=AUTOTUNE)
# val_ds = val_ds.prefetch(buffer_size=AUTOTUNE)
test_ds= test_ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
tf.get_logger().setLevel('ERROR')

In [34]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [29]:
def build_model(hp):
    model = tf.keras.models.load_model("Models/ResNet50-STBv1.0_16")
    
    # Setting Resnet50 to be false
    for layer in model.layers[2].layers:
        layer.trainable = False

    if hp.Choice("conv1",[True,False]):
        for layer in model.layers[2].layers[1:7]:
            layer.trainable = True
        
    if hp.Choice("conv2_block1",[True,False]):
        for layer in model.layers[2].layers[7:19]:
            layer.trainable = True
    
    if hp.Choice("conv2_block2",[True,False]):
        for layer in model.layers[2].layers[19:29]:
            layer.trainable = True

    if hp.Choice("conv2_block3",[True,False]):
        for layer in model.layers[2].layers[29:39]:
            layer.trainable = True
    
    if hp.Choice("conv3_block1",[True,False]):
        for layer in model.layers[2].layers[39:51]:
            layer.trainable = True

    if hp.Choice("conv3_block2",[True,False]):
        for layer in model.layers[2].layers[51:61]:
            layer.trainable = True
    
    if hp.Choice("conv3_block3",[True,False]):
        for layer in model.layers[2].layers[61:71]:
            layer.trainable = True

    if hp.Choice("conv3_block4",[True,False]):
        for layer in model.layers[2].layers[71:81]:
            layer.trainable = True

    # if hp.Choice("conv4_block1",[True,False]):
    #     for layer in model.layers[2].layers[81:93]:
    #         layer.trainable = True
    
    # if hp.Choice("conv4_block2",[True,False]):
    #     for layer in model.layers[2].layers[93:103]:
    #         layer.trainable = True
    
    # if hp.Choice("conv4_block3",[True,False]):
    #     for layer in model.layers[2].layers[103:113]:
    #         layer.trainable = True
    
    # if hp.Choice("conv4_block4",[True,False]):
    #     for layer in model.layers[2].layers[113:123]:
    #         layer.trainable = True
    
    # if hp.Choice("conv4_block5",[True,False]):
    #     for layer in model.layers[2].layers[123:133]:
    #         layer.trainable = True
    
    # if hp.Choice("conv4_block6",[True,False]):
    #     for layer in model.layers[2].layers[133:143]:
    #         layer.trainable = True
    
    # if hp.Choice("conv5_block1",[True,False]):
    #     for layer in model.layers[2].layers[143:155]:
    #         layer.trainable = True

    # if hp.Choice("conv5_block2",[True,False]):
    #     for layer in model.layers[2].layers[155:165]:
    #         layer.trainable = True
    
    # if hp.Choice("conv5_block3",[True,False]):
    #     for layer in model.layers[2].layers[165:]:
    #         layer.trainable = True
    
    model.compile(optimizer=tf.keras.optimizers.SGD(lr=1e-3, momentum=0.9),loss=tf.keras.losses.SparseCategoricalCrossentropy(),metrics=['accuracy'])
    return model

In [ ]:
%pip install keras_tuner

In [25]:
from keras_tuner.engine.hyperparameters import HyperParameters

# Define the hyperparameters
hp = HyperParameters()

In [ ]:
%pip install keras-tuner --upgrade

In [26]:
from keras_tuner import Hyperband, Objective

In [55]:
tuner = Hyperband(
    build_model,
    objective=Objective('val_accuracy', direction='max'),
    max_epochs=25,
    factor=3,
    overwrite=True,
    directory='Tuner/ResNet50-STBv1.0_17',
    project_name='HB_tuner')

In [56]:
tuner.search_space_summary(extended=False)

Search space summary
Default search space size: 8
conv1 (Choice)
{'default': 1, 'conditions': [], 'values': [1, 0], 'ordered': True}
conv2_block1 (Choice)
{'default': 1, 'conditions': [], 'values': [1, 0], 'ordered': True}
conv2_block2 (Choice)
{'default': 1, 'conditions': [], 'values': [1, 0], 'ordered': True}
conv2_block3 (Choice)
{'default': 1, 'conditions': [], 'values': [1, 0], 'ordered': True}
conv3_block1 (Choice)
{'default': 1, 'conditions': [], 'values': [1, 0], 'ordered': True}
conv3_block2 (Choice)
{'default': 1, 'conditions': [], 'values': [1, 0], 'ordered': True}
conv3_block3 (Choice)
{'default': 1, 'conditions': [], 'values': [1, 0], 'ordered': True}
conv3_block4 (Choice)
{'default': 1, 'conditions': [], 'values': [1, 0], 'ordered': True}


In [31]:
# create a log directory for TensorBoard
log_dir = "logs/ResNet50-STBv1.0_17"

# create a TensorBoard callback to visualize training metrics
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [47]:
from datetime import datetime

In [62]:
# Create the TrialCheckpoint callback
class MyModelCheckpoint (ModelCheckpoint):
    def __init__(self, *args, **kwargs):
        super(MyModelCheckpoint, self).__init__(
            filepath = self.get_chpt_dir(),
            save_weights_only=False,
            save_best_only=True,
            save_freq='epoch',
            verbose=1
        )

    def get_now(self):
        now = datetime.now()
        now = now.strftime("%Y%m%d%H%M%S")
        return now

    
    def get_chpt_dir(self):
        return f"Checkpoint/ResNet50-STBv1.0_17/{trial.trial_id}{self.get_now().}"

In [60]:
f = tf.keras.models.load_model("Tuner/ResNet50-STBv1.0_7/resnet50_stb_v1_0_7.02.h5")

In [61]:
f.evaluate(test_ds)

11/11 [==============================] - 1s 29ms/step - loss: 2.2109 - accuracy: 0.4491


[2.210906744003296, 0.4491018056869507]

In [ ]:
all_models = tuner.get_best_models(num_models=28)

In [57]:
tuner.search(train_ds, epochs=25, validation_data=test_ds,callbacks=[MyModelCheckpoint(), tensorboard_callback])

3916/3916 [==============================] - ETA: 0s - loss: 1.6222e-04 - accuracy: 0.9999
Epoch 16: val_loss did not improve from 1.69591
3916/3916 [==============================] - 351s 90ms/step - loss: 1.6222e-04 - accuracy: 0.9999 - val_loss: 2.3715 - val_accuracy: 0.7006
Epoch 17/25
3916/3916 [==============================] - ETA: 0s - loss: 2.8864e-05 - accuracy: 1.0000
Epoch 17: val_loss did not improve from 1.69591
3916/3916 [==============================] - 350s 89ms/step - loss: 2.8864e-05 - accuracy: 1.0000 - val_loss: 2.5266 - val_accuracy: 0.6886
Epoch 18/25
2881/3916 [=====================>........] - ETA: 1:32 - loss: 1.8896e-04 - accuracy: 1.0000

KeyboardInterrupt: 

## Evaluation and Predictions

In [ ]:
model = tf.keras.models.load_model("Models/ResNet50-STBv1.0_6")

In [70]:
test_dir = pathlib.Path("dcrpi-167")
image_height, image_width = 224, 224
# with tf.device("CPU"):
test_ds = tf.keras.utils.image_dataset_from_directory(
  test_dir,
  seed=123,
  image_size=(image_height, image_width),
  batch_size=1
  )

Found 167 files belonging to 3 classes.


In [ ]:
test_ds = test_ds.map(to_one_hot)

In [ ]:
loss, precision = model.evaluate(test_ds)

In [64]:
num_models = 28
all_models = tuner.get_best_models(num_models=num_models)

In [71]:
tuner_results = {}
for i,model in enumerate(all_models):
    print(f"{i}-best model")
    l,p = model.evaluate(test_ds)
    tuner_results[i] = [l,p]
    

0-best model


2023-04-10 01:40:56.818629: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_4' with dtype int32 and shape [167]
	 [[{{node Placeholder/_4}}]]
2023-04-10 01:40:56.818868: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'Placeholder/_0' with dtype string and shape [167]
	 [[{{node Placeholder/_0}}]]


167/167 [==============================] - 2s 7ms/step - loss: 1.9320 - accuracy: 0.3832
1-best model
167/167 [==============================] - 2s 6ms/step - loss: 1.6627 - accuracy: 0.7545
2-best model
167/167 [==============================] - 2s 6ms/step - loss: 1.9213 - accuracy: 0.7545
3-best model
167/167 [==============================] - 2s 6ms/step - loss: 1.3871 - accuracy: 0.7425
4-best model
167/167 [==============================] - 2s 6ms/step - loss: 1.9280 - accuracy: 0.7485
5-best model
167/167 [==============================] - 2s 6ms/step - loss: 1.8094 - accuracy: 0.7425
6-best model
167/167 [==============================] - 2s 6ms/step - loss: 1.9033 - accuracy: 0.7365
7-best model
167/167 [==============================] - 2s 6ms/step - loss: 1.9026 - accuracy: 0.7365
8-best model
167/167 [==============================] - 2s 6ms/step - loss: 1.6752 - accuracy: 0.7365
9-best model
167/167 [==============================] - 2s 6ms/step - loss: 1.7715 - accuracy: 

In [72]:
best_model_1 = all_models[1]
best_model_1.save("Models/ResNet50-STBv1.0_17")

2023-04-10 01:43:38.698990: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'random_flip_input' with dtype float and shape [?,224,224,3]
	 [[{{node random_flip_input}}]]
2023-04-10 01:43:38.747680: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype float and shape [?,224,224,3]
	 [[{{node inputs}}]]
2023-04-10 01:43:38.755194: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'random_flip_input' with dtype float and shape [

INFO:tensorflow:Assets written to: Models/ResNet50-STBv1.0_17/assets


INFO:tensorflow:Assets written to: Models/ResNet50-STBv1.0_17/assets


In [ ]:
import json
with open('tuner_result.json', 'w') as fp:
    json.dump(tuner_results, fp)

In [ ]:
num_trials = 15
from kerastuner import HyperParameters
tuner_hps_summary = tuner.get_best_hyperparameters(num_trials=num_trials)

In [ ]:
summary_dict = {}
for i,a in enumerate(tuner_hps_summary):
  print(f"Best trial {i}:")

  trial_dict = {}
  for hp_name, hp_value in a.values.items():
      # print(f"{hp_value}")
      trial_dict[hp_name] = hp_value
  summary_dict[i] = trial_dict

In [ ]:
import pandas as pd

# Create a pandas DataFrame from the dictionary
df = pd.DataFrame.from_dict(summary_dict, orient='index')

# Write the DataFrame to an excel file
df.to_excel('/content/drive/My Drive/Models/output-3_1.xlsx', na_rep='')

In [ ]:
print("Best hyperparameters:")

for i,trial in enumerate(tuner_hps_summary):
  print(f"Trial {i}")
  for hp_name, hp_value in trial.values.items():
      print(f"- {hp_name}: {hp_value}")

## Convert to TfLite

In [ ]:
model = keras.models.load_model("Models/ResNet50-STBv1.0_13")

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('Models/ResNet50-STBv1.0_13.Lite', 'wb') as f:
  f.write(tflite_model)

In [ ]:
model.summary()